In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
import pandas as pd #collection of functions for data processing and 
# analysis modeled after R dataframes with SQL like features
import matplotlib #collection of functions for scientific and publication-ready visualization
import numpy as np #foundational package for scientific computing
import scipy as sp #collection of functions for scientific computing and advance mathematics
# numpyより高度なことができる。フーリエ変換とかね。
import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
# https://qiita.com/5t111111/items/7852e13ace6de288042f
# ipythonは便利なシェル？
import sklearn #collection of machine learning algorithms
# misc libraries
# mist = その他
import random
import time
# ignore warning
import warnings
warnings.filterwarnings('ignore')
# よくわからん原因不明の警告が表示されることがあるので、それを防止できる。

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# Any results you write to the current directory are saved as output.

In [3]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')  # なんかよくわからんが、プロットのスタイル
sns.set_style('white')  # 背景白、グリッドなし
pylab.rcParams['figure.figsize'] = 12,8  # matplotの調整

In [6]:
#import data from file: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
data_raw = pd.read_csv('../input/train.csv')
print('--------train data---------')
print(data_raw)
#a dataset should be broken into 3 splits: train, test, and (final) validation
#the test file provided is the validation file for competition submission
#we will split the train set into train and test data in future sections
data_val = pd.read_csv('../input/test.csv')
print('--------test data---------')
print(data_val)
#to play with our data we'll create a copy
#remember python assignment or equal passes by reference vs values,
# so we use the copy function: 
# https://stackoverflow.com/questions/46327494/python-pandas-dataframe-copydeep-false-vs-copydeep-true-vs
data1 = data_raw.copy(deep = True)
# http://kurochan-note.hatenablog.jp/entry/20110316/1300267023
print('--------data1---------')
print(data1)

#however passing by reference is convenient, because we can clean both datasets at once
data_cleaner = [data1, data_val]
# 一度にデータがそうさできて楽なんで、まとめる。
print('--------data_cleaner---------')
print(data_cleaner)

#preview data
print('---------raw info----------')
print (data_raw.info()) 
print('---------samples----------')
data_raw.sample(10) #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html

--------train data---------
     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
5              6         0       3   
6              7         0       1   
7              8         0       3   
8              9         1       3   
9             10         1       2   
10            11         1       3   
11            12         1       1   
12            13         0       3   
13            14         0       3   
14            15         0       3   
15            16         1       2   
16            17         0       3   
17            18         1       2   
18            19         0       3   
19            20         1       3   
20            21         0       2   
21            22         1       2   
22            23         1       3   
23            24         1       1   
24            25      

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
232,233,0,2,"Sjostedt, Mr. Ernst Adolf",male,59.0,0,0,237442,13.5000,NaN,S
356,357,1,1,"Bowerman, Miss. Elsie Edith",female,22.0,0,1,113505,55.0000,E33,S
86,87,0,3,"Ford, Mr. William Neal",male,16.0,1,3,W./C. 6608,34.3750,NaN,S
821,822,1,3,"Lulic, Mr. Nikola",male,27.0,0,0,315098,8.6625,NaN,S
120,121,0,2,"Hickman, Mr. Stanley George",male,21.0,2,0,S.O.C. 14879,73.5000,NaN,S
336,337,0,1,"Pears, Mr. Thomas Clinton",male,29.0,1,0,113776,66.6000,C2,S
634,635,0,3,"Skoog, Miss. Mabel",female,9.0,3,2,347088,27.9000,NaN,S
475,476,0,1,"Clifford, Mr. George Quincy",male,NaN,0,0,110465,52.0000,A14,S
35,36,0,1,"Holverson, Mr. Alexander Oskar",male,42.0,1,0,113789,52.0000,NaN,S


In [12]:
# nullを見に行く
# print('-'*12, 'Train:\n', data1.isnull())  # 中身がnullか調べて、nullならtrue 
print('Train column has with null values:\n', data1.isnull().sum())
# .sum()で、各columsごとに、Trueの数を数えてる
print('-'*10)
print('Test column has with null values:\n', data_val.isnull().sum())
# .sum()で、各columsごとに、Trueの数を数えてる
print('-'*10)


Train column has with null values:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------
Test column has with null values:
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
----------


In [15]:
data_raw.describe(include = 'all')
# uniqueはデータの種類(bool)
# top：最頻値
# uniqueとかtopはboolにのみ作用できる。数値データとかは無理ぽい
# freque：最も一般的な値のでる回数

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Elias, Mr. Dibo",male,NaN,NaN,NaN,CA. 2343,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [22]:
# age, cabin, embarkedの欠損を埋める
# ageは数値データ、cabinはcategorical
# age, embarkedを埋める。cabinは消す.cabinは欠損が多すぎる。
# print(data1['Cabin'])
# print(data1['Embarked'])
for dataset in data_cleaner:
    dataset['Age'].fillna(dataset['Age'].median(), inplace =  True)
    # inpulaceで元のオブジェクト自体が変更される。
    dataset['Embarked'].fillna(dataset['Embarked'].mode(), inplace = True)
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
data_column = ['Passengers', 'Cabin', 'Ticket']
data1.drop(data_column, axis=1,  inplace = True)
#dropはindexを指定して削除できる。axis=1だと列で削除



ValueError: labels ['Passengers'] not contained in axis

In [24]:
data_raw.describe(include = 'all')


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Elias, Mr. Dibo",male,NaN,NaN,NaN,CA. 2343,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [25]:
print(data1)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
5              6         0       3   
6              7         0       1   
7              8         0       3   
8              9         1       3   
9             10         1       2   
10            11         1       3   
11            12         1       1   
12            13         0       3   
13            14         0       3   
14            15         0       3   
15            16         1       2   
16            17         0       3   
17            18         1       2   
18            19         0       3   
19            20         1       3   
20            21         0       2   
21            22         1       2   
22            23         1       3   
23            24         1       1   
24            25         0       3   
25          

In [29]:
###CREATE: Feature Engineering for train and test/validation dataset
for dataset in data_cleaner:
    dataset['Familysize'] = dataset['SibSp'] + dataset['SibSp'] + 1
    dataset['IsAlone'] = 1 # initialize by 1
    dataset['IsAlone'].loc[dataset['Familysize'] > 1] = 0
    # loc[]なのに注意参照元.loc[指定したい条件]
    # loc[]で、条件に合うリストをとってこれる
    dataset['Title'] = dataset['Name'].str.split(",", expand=True)[1].str.split(".", expand=True)[0]
    # pandasはstr型のデータに当たるとき、.str＝strアクセを使うと各データに対して処理を適用できる
    # expand=Trueで複数の列に分割できる
    # qcut, cutは連続値を任意の境界で区分けして離散値にする。=ビニング処理をする
    # qcutはビンに含まれる要素数を等しくビニング処理
    # cutは最大値と最小値の間で値の間隔が等しくなるよう分割
    dataset['Farebin'] = pd.qcut(dataset['Fare'], 4)
    # astypeでpandasの中身をキャストできる
    dataset['Agebin'] = pd.cut(dataset['Age'].astype(int), 5)

In [32]:
  #cleanup rare title names
stat_min = 10
# print(data1['Title'].value_counts())
title_name = (data1['Title'].value_counts() < stat_min)
# 各タイトルに対して、出現数をカウントしたリストが生成され、その各要素に対して、判定
# print(title_name)


In [34]:
# rareなタイトルなら、その他=Miscとする。
data1['Title'] = data1['Title'].apply(lambda x: 'Misc' if title_name.loc[x] == True else x)
# apply()各列に関数を適用

In [36]:
print(data1['Title'].value_counts())

 Mr        517
 Miss      182
 Mrs       125
 Master     40
Misc        27
Name: Title, dtype: int64


In [37]:
data1.info()
# 「有効データ数」「データ型」「メモリ使用量」などの総合的な情報を表示
# なんかよくわからんがobjectはstrでcategoryは値の範囲という理解にしとこう。無理。

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 18 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
Family_size    891 non-null int64
IsAlone        891 non-null int64
Familysize     891 non-null int64
Title          891 non-null object
Farebin        891 non-null category
Agebin         891 non-null category
dtypes: category(2), float64(2), int64(8), object(6)
memory usage: 113.3+ KB


In [42]:
print(data1['Title'],data1['Cabin'],  data1['Farebin'])

0           Mr
1          Mrs
2         Miss
3          Mrs
4           Mr
5           Mr
6           Mr
7       Master
8          Mrs
9          Mrs
10        Miss
11        Miss
12          Mr
13          Mr
14        Miss
15         Mrs
16      Master
17          Mr
18         Mrs
19         Mrs
20          Mr
21          Mr
22        Miss
23          Mr
24        Miss
25         Mrs
26          Mr
27          Mr
28        Miss
29          Mr
        ...   
861         Mr
862        Mrs
863       Miss
864         Mr
865        Mrs
866       Miss
867         Mr
868         Mr
869     Master
870         Mr
871        Mrs
872         Mr
873         Mr
874        Mrs
875       Miss
876         Mr
877         Mr
878         Mr
879        Mrs
880        Mrs
881         Mr
882       Miss
883         Mr
884         Mr
885        Mrs
886       Misc
887       Miss
888       Miss
889         Mr
890         Mr
Name: Title, Length: 891, dtype: object 0              NaN
1              C85
2      

In [43]:
data_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 16 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
Familysize     418 non-null int64
IsAlone        418 non-null int64
Title          418 non-null object
Farebin        418 non-null category
Agebin         418 non-null category
dtypes: category(2), float64(2), int64(6), object(6)
memory usage: 46.8+ KB


In [44]:
data1.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family_size,IsAlone,Familysize,Title,Farebin,Agebin
352,353,0,3,"Elias, Mr. Tannous",male,15.0,1,1,2695,7.2292,NaN,C,3,0,3,Mr,"(-0.001, 7.91]","(-0.08, 16.0]"
716,717,1,1,"Endres, Miss. Caroline Louise",female,38.0,0,0,PC 17757,227.5250,C45,C,1,1,1,Miss,"(31.0, 512.329]","(32.0, 48.0]"
34,35,0,1,"Meyer, Mr. Edgar Joseph",male,28.0,1,0,PC 17604,82.1708,NaN,C,3,0,3,Mr,"(31.0, 512.329]","(16.0, 32.0]"
93,94,0,3,"Dean, Mr. Bertram Frank",male,26.0,1,2,C.A. 2315,20.5750,NaN,S,3,0,3,Mr,"(14.454, 31.0]","(16.0, 32.0]"
427,428,1,2,"Phillips, Miss. Kate Florence (""Mrs Kate Louis...",female,19.0,0,0,250655,26.0000,NaN,S,1,1,1,Miss,"(14.454, 31.0]","(16.0, 32.0]"
25,26,1,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38.0,1,5,347077,31.3875,NaN,S,3,0,3,Mrs,"(31.0, 512.329]","(32.0, 48.0]"
484,485,1,1,"Bishop, Mr. Dickinson H",male,25.0,1,0,11967,91.0792,B49,C,3,0,3,Mr,"(31.0, 512.329]","(16.0, 32.0]"
105,106,0,3,"Mionoff, Mr. Stoytcho",male,28.0,0,0,349207,7.8958,NaN,S,1,1,1,Mr,"(-0.001, 7.91]","(16.0, 32.0]"
388,389,0,3,"Sadlier, Mr. Matthew",male,28.0,0,0,367655,7.7292,NaN,Q,1,1,1,Mr,"(-0.001, 7.91]","(16.0, 32.0]"
760,761,0,3,"Garfirth, Mr. John",male,28.0,0,0,358585,14.5000,NaN,S,1,1,1,Mr,"(14.454, 31.0]","(16.0, 32.0]"


In [54]:
drop_column = ['PassengerId', 'Cabin', 'Ticket']
# cagin=客室
data1.drop(drop_column, axis=1, inplace=True)

In [51]:
# sklearnとpandasを使って、カテゴリカルをダミーに
print(data_cleaner)

[     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
5              6         0       3   
6              7         0       1   
7              8         0       3   
8              9         1       3   
9             10         1       2   
10            11         1       3   
11            12         1       1   
12            13         0       3   
13            14         0       3   
14            15         0       3   
15            16         1       2   
16            17         0       3   
17            18         1       2   
18            19         0       3   
19            20         1       3   
20            21         0       2   
21            22         1       2   
22            23         1       3   
23            24         1       1   
24            25         0       3   
25         

In [55]:
data1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Embarked       889 non-null object
Family_size    891 non-null int64
IsAlone        891 non-null int64
Familysize     891 non-null int64
Title          891 non-null object
Farebin        891 non-null category
Agebin         891 non-null category
dtypes: category(2), float64(2), int64(7), object(4)
memory usage: 92.5+ KB


In [53]:
# sex、embarked、title(object)と、farebin、Agebin(category)を変換
label = LabelEncoder()
for dataset in data_cleaner:
    dataset['Sex_Code'] = 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family_size,IsAlone,Familysize,Title,Farebin,Agebin
475,476,0,1,"Clifford, Mr. George Quincy",male,28.0,0,0,110465,52.0,A14,S,1,1,1,Mr,"(31.0, 512.329]","(16.0, 32.0]"
